In [1]:
import os
from fnmatch import fnmatch
from trainData_featExt_glcm import trainData_featExt_glcm
from trainData_featExt_gabor import trainData_featExt_gabor
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.glcm import Glcm
from feature_extraction.gabor import Gabor
import sklearn
import time
from gmm import Gmm
from sklearn import preprocessing
import pickle

In [3]:
#load training samples feature vector
train_samples_feats_matrix_Gabor = np.load("train_samples_feats_matrix_Gabor.npy")

In [2]:
#load training samples feature vector
valid_samples_feats_matrix_Gabor = np.load("valid_samples_feats_matrix_Gabor.npy")

In [4]:
test_samples_feats_matrix_Gabor = np.load("test_samples_feats_matrix_Gabor.npy")

In [ ]:
#Train a gaussian mixture model from training data
t0 = time.time()
gm_model = sklearn.mixture.GaussianMixture(n_components=1000, covariance_type='full', max_iter = 300)
gm_model.fit(train_samples_feats_matrix_Gabor)
print("traing time: ", time.time() - t0)

In [5]:
gm_model = pickle.load( open( 'GMM_Gabor_1000', "rb" ))

In [6]:
#Predict labels of the new data points
y_predict = gm_model.predict(valid_samples_feats_matrix_Gabor)
#Predict novelty score of new data points
y_predict_score = gm_model.score_samples(valid_samples_feats_matrix_Gabor)
tmp = []
#novelty_thresh = gm_model.score(train_samples_feats_matrix_LBP)
x_predict_score = gm_model.score_samples(train_samples_feats_matrix_Gabor)
novelty_thresh = np.amin(x_predict_score)

# if data point belongs to the model then 1 else 0
#tmp = np.zeros_like(y_predict_score)
tmp = np.repeat(1, 10000)
#74.2344062054
tmp[y_predict_score < novelty_thresh] = 0


In [11]:
#Predict novelty score of new data points
test_y_predict_score = gm_model.score_samples(test_samples_feats_matrix_Gabor)
tmp2 = []
#novelty_thresh = gm_model.score(train_samples_feats_matrix_LBP)
#x_predict_score = gm_model.score_samples(train_samples_feats_matrix_Gabor)
#novelty_thresh = np.amin(x_predict_score)

# if data point belongs to the model then 1 else 0
#tmp = np.zeros_like(y_predict_score)
tmp2 = np.repeat(1, 10000)
#74.2344062054
tmp2[test_y_predict_score < 74.2344062054] = 0

In [7]:
#Ground truth
y1 = np.repeat(1, 6000) #normal
y2 = np.repeat(0, 4000) #abnormal
y = np.concatenate((y1,y2))

In [ ]:
f1_binary = sklearn.metrics.f1_score(y, tmp, pos_label = 0, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, tmp, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, tmp) 
kappa = sklearn.metrics.cohen_kappa_score(y, tmp)
print("kappa ", kappa)
tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(y, tmp).ravel()
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)
print("confusion matrix", tn, fp, fn, tp)

In [ ]:
#Get different thresholds value from ROC with corresponding F1_score and AUC
fpr,tpr,thresh = sklearn.metrics.roc_curve(y, y_predict_score)
for t, thres in enumerate (thresh) :
    tmp = np.repeat(1,10000)
    tmp[y_predict_score < thres] = 0
    f1 = sklearn.metrics.f1_score(y, tmp, average = 'macro')
    print ("f1: ", f1, "thres: ", thres)

In [18]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io


fpr,tpr,thresh = sklearn.metrics.roc_curve(y,y_predict_score)
test_fpr,test_tpr,test_thresh = sklearn.metrics.roc_curve(y,test_y_predict_score)
#print ('true positive rate: ',tpr )
#print ('true positive rate: ',1 - fpr )
f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="GMM-Validation")
ax.plot(test_fpr,test_tpr,label="GMM-test")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")

# these are matplotlib.patch.Patch properties
#props = dict(boxstyle='square', facecolor='white', alpha=0.5)

#textstr = '$F1-binary=%.2f$\n$F1-macro=%.2f$\n$math-corcoeffient=%.2f$\n$AUC=%.2f$'%(0.66, 0.74, 0.51, 0.83)
# place a text box in upper left in axes coords
#ax.text(0.95, 0.2, textstr, transform=ax.transAxes, fontsize=14,
#    horizontalalignment='right', verticalalignment='bottom', bbox=props)

io.show()

UsageError: unrecognized arguments: off


In [14]:
f.savefig('GMM_Gabor_test-vs-validation.png')